In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

In [4]:
from get_data import load_history, load_current, combine_history, process_y, load_bios, load_team_data
# load_history()
# load_current()
# combine_history()
# load_team_data()
# load_bios()
# process_y()

from model_training import load_feature_maps, run_training

full_features_map, player_features_map = load_feature_maps()
pipes = run_training('skater', full_features_map, player_features_map, retrain=True)

goalie_features_map, g_player_features_map = load_feature_maps('goalie')
g_pipes = run_training('goalie', goalie_features_map, g_player_features_map, retrain=True)

import pickle
pickle.dump(pipes, open('data/skater_models.p', 'wb'))
pickle.dump(g_pipes, open('data/goalie_models.p', 'wb'))

g -- 0.08215589819597358
g -- 0.2691759409542468
a -- 0.0928994577565353
a -- 0.40650624968642257
sog -- 0.23158330461124488
sog -- 1.06760539188018
fow -- 0.7523712678711754
fow -- 0.7787308107006455
hit -- 0.2683402959400334
hit -- 0.9725752470849486
block -- 0.23555499964492932
block -- 0.7609384142545998
pim -- 0.02597477820674554
pim -- 0.706147428666212
goalsfor -- 0.05142459596541038
goalsfor -- 0.6330227565045711
goalsaga -- 0.03328360085634807
goalsaga -- 0.6266593657093465
ppp -- 0.09983279086078412
ppp -- 0.15502061060893887
ga -- 0.13166408400883123
ga -- 1.1629838282921716
win -- 0.07687043104624058
win -- 0.2578682460754789
so -- 0.18294616861468516
so -- 0.3725868408276006
save -- 0.17484469063985064
save -- 9.844253562557185
icetime -- 0.19608094919618224
icetime -- 1291.8171405179978


/opt/conda/envs/yh/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.908e+10, tolerance: 4.822e+06
  model = cd_fast.enet_coordinate_descent(
/opt/conda/envs/yh/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.240e+10, tolerance: 5.646e+06
  model = cd_fast.enet_coordinate_descent(


NameError: name 'players' is not defined